In [1]:
from sodapy import Socrata
import requests, zipfile, StringIO
import urllib2
import pandas as pd
import numpy as np
import pypyodbc
import csv
import os
import tempfile
import statsmodels.formula.api as smf
import pylab as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
puma_ref = pd.read_excel('http://www.nyc.gov/html/dcp/download/census/nyc2010census_tabulation_equiv.xlsx',header=3)
# alternatively, use shapefile from https://data.cityofnewyork.us/Housing-Development/Public-Use-Microdata-Areas-PUMA-/cwiz-gcty
puma_list = puma_ref['PUMA'].unique()[~np.isnan(puma_ref['PUMA'].unique())]

In [3]:
pums_data = pd.read_csv('ss14hny.csv')
pums_data['PUMA'] = pums_data[['PUMA00','PUMA10']].max(axis=1)

In [4]:
nyc_pums = pd.DataFrame()
for puma in puma_list:
    nyc_pums = nyc_pums.append(pums_data.query('PUMA == @puma'))
    print nyc_pums.shape[0],'rows after addition of PUMA',str(puma)

2532 rows after addition of PUMA 3704.0
4290 rows after addition of PUMA 3706.0
7271 rows after addition of PUMA 3705.0
9141 rows after addition of PUMA 3703.0
11081 rows after addition of PUMA 3708.0
13603 rows after addition of PUMA 3702.0
15801 rows after addition of PUMA 3707.0
18926 rows after addition of PUMA 3710.0
21257 rows after addition of PUMA 3701.0
24034 rows after addition of PUMA 3709.0
27707 rows after addition of PUMA 4017.0
30805 rows after addition of PUMA 4013.0
33408 rows after addition of PUMA 4003.0
36267 rows after addition of PUMA 4014.0
38410 rows after addition of PUMA 4018.0
41972 rows after addition of PUMA 4004.0
44269 rows after addition of PUMA 4007.0
46446 rows after addition of PUMA 4002.0
50660 rows after addition of PUMA 4009.0
53223 rows after addition of PUMA 4005.0
55807 rows after addition of PUMA 4006.0
57735 rows after addition of PUMA 4011.0
60478 rows after addition of PUMA 4008.0
63099 rows after addition of PUMA 4010.0
66109 rows after add

In [5]:
# excluding group quarters and vacant household records
nyc_pums = nyc_pums.query('TYPE == 1 & BLD > 1 & BLD < 10')
nyc_pums = nyc_pums[nyc_pums['VACS'].isnull()]

In [7]:
# can we identify renters who live in condo buildings?
nyc_pums.query('TEN == 3 and FCONP == 1')
# it appears not

,insp,RT,SERIALNO,DIVISION,PUMA00,PUMA10,REGION,ST,ADJHSG,ADJINC,...,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80,PUMA


In [8]:
# nyc_pums[nyc_pums['PUMA']==4004].groupby(['YBL']).size()

In [9]:
print('Example median HH income in Williamsburg/Greenpoint')
nyc_pums[nyc_pums['PUMA10']==4001]['HINCP'].median()

Example median HH income in Williamsburg/Greenpoint


60000.0

In [10]:
print ('Median HH income in all NYC PUMAs')
nyc_pums['HINCP'].median()

Median HH income in all NYC PUMAs


53600.0

In [11]:
# nyc_pums.groupby(['YBL'])['HINCP'].median()

In [12]:
# dependents to test: HINCP, RNTP, GRNTP, NPF, TAXP
# independents to test: YBL (ord), TEN (c), BDSP, BLD, VACS - perhaps as a filter?
# also of interest: MV

In [13]:
# model rent paid for renters
mod = smf.ols(formula='RNTP ~ BLD + BDSP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                   RNTP   R-squared:                       0.295
Model:                            OLS   Adj. R-squared:                  0.295
Method:                 Least Squares   F-statistic:                     457.9
Date:                Thu, 25 Feb 2016   Prob (F-statistic):               0.00
Time:                        13:36:58   Log-Likelihood:            -6.1681e+05
No. Observations:               78742   AIC:                         1.234e+06
Df Residuals:                   78669   BIC:                         1.234e+06
Df Model:                          72                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept         489.9351    431.965     

In [14]:
# model income for renters
mod = smf.ols(formula='HINCP ~ BLD + BDSP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                  HINCP   R-squared:                       0.173
Model:                            OLS   Adj. R-squared:                  0.172
Method:                 Least Squares   F-statistic:                     228.1
Date:                Thu, 25 Feb 2016   Prob (F-statistic):               0.00
Time:                        13:37:50   Log-Likelihood:            -9.9827e+05
No. Observations:               78742   AIC:                         1.997e+06
Df Residuals:                   78669   BIC:                         1.997e+06
Df Model:                          72                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept        3.745e+04   5.49e+04     

In [15]:
# model income for owners
mod = smf.ols(formula='HINCP ~ BLD + BDSP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']<3]).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                  HINCP   R-squared:                       0.197
Model:                            OLS   Adj. R-squared:                  0.196
Method:                 Least Squares   F-statistic:                     175.1
Date:                Thu, 25 Feb 2016   Prob (F-statistic):               0.00
Time:                        13:39:27   Log-Likelihood:            -6.6648e+05
No. Observations:               50835   AIC:                         1.333e+06
Df Residuals:                   50763   BIC:                         1.334e+06
Df Model:                          71                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept        1.602e+04   8.47e+04     

In [18]:
# model income for owners with property tax as independent, but without bedrooms or size of building (covairance)
mod = smf.ols(formula='HINCP ~ TAXP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']<3]).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                  HINCP   R-squared:                       0.177
Model:                            OLS   Adj. R-squared:                  0.176
Method:                 Least Squares   F-statistic:                     156.0
Date:                Thu, 25 Feb 2016   Prob (F-statistic):               0.00
Time:                        13:42:53   Log-Likelihood:            -6.6710e+05
No. Observations:               50835   AIC:                         1.334e+06
Df Residuals:                   50764   BIC:                         1.335e+06
Df Model:                          70                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept         8.59e+04   8.57e+04     

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
plt.hist(nyc_pums['VALP'],bins=20,range=(1,6000000))
plt.title('Histogram of Reported Property Values')
plt.ylabel('Frequency')
ax.xaxis.get_majorticklocs()